# 💸 FutureBank Sim – Colab Demo
Run this notebook to reproduce the simulation and figures.

In [ ]:
# Install minimal deps (Colab may already have some)
!pip -q install numpy pandas matplotlib pydantic


In [ ]:
# If running in Colab, mount Google Drive if you want to save outputs (optional)
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# --- Core modules (replicated inline for Colab demo) ---
from __future__ import annotations
import numpy as np
from typing import List, Dict
from pydantic import BaseModel, Field

class Goal(BaseModel):
    name: str
    year: int
    amount: float

class Loan(BaseModel):
    name: str
    principal: float
    annual_rate: float
    years: int
    def monthly_payment(self) -> float:
        r = self.annual_rate / 12.0
        n = self.years * 12
        if r == 0:
            return self.principal / n
        return self.principal * (r * (1 + r) ** n) / ((1 + r) ** n - 1)

class UserProfile(BaseModel):
    start_age: int = 21
    horizon_years: int = 10
    monthly_income: float = 40000.0
    monthly_expenses: float = 20000.0
    savings_rate: float = 0.35
    inflation: float = 0.06
    tax_rate: float = 0.05
    goals: List[Goal] = Field(default_factory=list)
    loans: List[Loan] = Field(default_factory=list)

class PortfolioConfig(BaseModel):
    expected_return: float = 0.12
    volatility: float = 0.18

class SimulationConfig(BaseModel):
    n_paths: int = 10000
    seed: int = 42

def simulate(profile: UserProfile, portfolio: PortfolioConfig, cfg: SimulationConfig):
    rng = np.random.default_rng(cfg.seed)
    months = profile.horizon_years * 12
    mu_m = (1 + portfolio.expected_return) ** (1/12) - 1
    sigma_m = portfolio.volatility / np.sqrt(12)
    monthly_loan = sum(l.monthly_payment() for l in profile.loans)

    wealth_paths = []
    times = list(range(months + 1))
    for p in range(cfg.n_paths):
        wealth, path = 0.0, [0.0]
        inc = profile.monthly_income
        exp = profile.monthly_expenses
        for t in range(1, months+1):
            shock = rng.normal(mu_m, sigma_m)
            tax = profile.tax_rate * inc
            net = inc - tax
            savings = profile.savings_rate * max(net - exp - monthly_loan, 0)
            wealth = max(0.0, wealth)*(1+shock) + savings
            # goals end of each goal year
            year = (t-1)//12 + 1
            for g in profile.goals:
                if g.year == year and (t % 12 == 0):
                    wealth -= g.amount
            if t % 12 == 0:
                inc *= (1+profile.inflation); exp *= (1+profile.inflation)
            path.append(wealth)
        wealth_paths.append(path)
    return np.array(wealth_paths), times


In [ ]:
# ---- Run a demo simulation ----
profile = UserProfile(
    start_age=21,
    horizon_years=10,
    monthly_income=45000,
    monthly_expenses=22000,
    savings_rate=0.35,
    inflation=0.06,
    tax_rate=0.05,
    goals=[Goal(name='Car', year=2, amount=700000),
           Goal(name='Masters', year=4, amount=2000000)],
    loans=[Loan(name='EduLoan', principal=500000, annual_rate=0.12, years=5)]
)
portfolio = PortfolioConfig(expected_return=0.12, volatility=0.18)
cfg = SimulationConfig(n_paths=5000, seed=42)
arr, times = simulate(profile, portfolio, cfg)

import numpy as np
percentiles = {k: np.percentile(arr, v, axis=0) for k,v in [('p5',5),('p25',25),('p50',50),('p75',75),('p95',95)]}
goal_success = {}
for g in profile.goals:
    idx = g.year*12
    goal_success[g.name] = float((arr[:, idx] >= 0).mean())

percentiles, goal_success


In [ ]:
# ---- Plot percentiles (matplotlib, no seaborn, single plot) ----
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,4))
for k in ['p5','p25','p50','p75','p95']:
    ax.plot(times, percentiles[k], label=k.upper())
ax.set_xlabel('Months'); ax.set_ylabel('Wealth')
ax.set_title('Wealth Over Time (Monte Carlo Percentiles)')
ax.legend(); plt.show()
